In [1]:

import pickle
import numpy as np
import os
#from scipy.misc import imread
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from sklearn.feature_extraction import image

from collections import Counter
from scipy.stats.mstats import mode

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = pickle.load(f,encoding='latin1')
    X = datadict['data']
    
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("double")
    Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
  """ load all of cifar """
  xs = []
  ys = []
  for b in range(1,6):
    f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
    X, Y = load_CIFAR_batch(f)
    xs.append(X)
    ys.append(Y)    
  Xtr = np.concatenate(xs)
  Ytr = np.concatenate(ys)
  del X, Y
  Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
#Normalise la base - la moyenne , divise par std, resultat final toujours mauvais normalisé ou non
#Normalize the data, by soustracting the mean and dividing the standart deviation 
  mean_image = np.mean(Xtr, axis=0)
  std_image=np.std(Xtr, axis=0)
  #Xtr -= mean_image
  #Xtr /= std_image
  
  #Xte -= mean_image
  #Xte /= std_image
  return Xtr, Ytr, Xte, Yte



def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    cifar10_dir = 'C:/Users/MyPC/Desktop/Projet Apprentissage Image/cifar-10-batches-py/'
    
    print (len(load_CIFAR10(cifar10_dir)))
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_train=X_train.swapaxes(1,3)
    X_val=X_val.swapaxes(1,3)
    X_test=X_test.swapaxes(1,3)
    return X_train, y_train, X_val, y_val, X_test, y_test


X_train, y_train, X_val, y_val, X_test, y_test=get_CIFAR10_data()


4


In [3]:
#appliquer directement des classifieurs sur 
XtrainBis= np.zeros((len(X_train),3072))
for i in range(0,len(X_train)):
    XtrainBis[i,:]=X_train[i,:].reshape(1,-1)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(XtrainBis, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [7]:
XtestBis= np.zeros((len(X_test),3072))
for i in range(0,len(X_test)):
    XtestBis[i,:]=X_test[i,:].reshape(1,-1)

In [11]:
y_pred= neigh.predict(XtestBis)

In [12]:
(y_pred==y_test).mean()

0.327

In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(XtrainBis, y_train)
y_pred1= neigh.predict(XtestBis)
(y_pred1==y_test).mean()

0.327

In [16]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0).fit(XtrainBis)
kmeans.labels_

y_pred2=kmeans.predict(XtestBis)
(y_pred2==y_test).mean()

0.076

In [15]:
import itertools
%matplotlib
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

y_test = y_test
y_pred=y_pred1 


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['airplane', 'automobile', 'bird', 'cat', 'deer' ,'dog', 'frog', 'horse' ,'ship' ,'truck'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['airplane', 'automobile', 'bird', 'cat', 'deer' ,'dog', 'frog', 'horse' ,'ship' ,'truck'], normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Using matplotlib backend: TkAgg
Confusion matrix, without normalization
[[52  1 12  1  5  0  5  3 23  1]
 [20 29 10  9  8  1  1  0 10  1]
 [18  0 40 10 17  6  5  1  3  0]
 [ 4  2 24 32 16  8 11  2  3  1]
 [10  0 17  5 39  5  7  2  5  0]
 [ 6  1 25 16 16 15  4  1  2  0]
 [ 7  1 35  7 37  1 22  1  1  0]
 [11  3 27  7 25  6  3 16  3  1]
 [18  5  2  4  6  1  0  1 69  0]
 [20  6 14  9 17  3  2  1 24 13]]
Normalized confusion matrix
[[0.5  0.01 0.12 0.01 0.05 0.   0.05 0.03 0.22 0.01]
 [0.22 0.33 0.11 0.1  0.09 0.01 0.01 0.   0.11 0.01]
 [0.18 0.   0.4  0.1  0.17 0.06 0.05 0.01 0.03 0.  ]
 [0.04 0.02 0.23 0.31 0.16 0.08 0.11 0.02 0.03 0.01]
 [0.11 0.   0.19 0.06 0.43 0.06 0.08 0.02 0.06 0.  ]
 [0.07 0.01 0.29 0.19 0.19 0.17 0.05 0.01 0.02 0.  ]
 [0.06 0.01 0.31 0.06 0.33 0.01 0.2  0.01 0.01 0.  ]
 [0.11 0.03 0.26 0.07 0.25 0.06 0.03 0.16 0.03 0.01]
 [0.17 0.05 0.02 0.04 0.06 0.01 0.   0.01 0.65 0.  ]
 [0.18 0.06 0.13 0.08 0.16 0.03 0.02 0.01 0.22 0.12]]


In [2]:
#create unsurpervised data
import numpy as np
patches=[]
rfSize = 6
numCentroids=100
whitening=True
numPatches = 200000

CIFAR_DIM=[32,32,3]
numPatches = 200000
rfSize = 6
patches= np.zeros((numPatches, rfSize*rfSize*3))
for i in range(0, numPatches):
    if i%1000==0:
        print('Extraction patche:', i, '/', numPatches)
    # selection d'un entier aléatoire entre 0 et 26 (pour pas dépasser le format image 32 par 32)
    r= np.random.randint(CIFAR_DIM[0]-rfSize) 
    c= np.random.randint(CIFAR_DIM[0]-rfSize)
    # on utilise ici le modulo pour extraire plusieurs patchs par image
    #(tous les len(X_train) un patche sera issu d'une même image 
    patch= (X_train[(i%len(X_train)),:])  
    
   #1ère dimension 3 , 2eme 32 , 3eme 32, ici taille du patch à taille voulut [3,6,6]

    patch= patch[:,r:r+rfSize, c:c+rfSize]
  #on reshape le patch pour qu'il soit un vecteut (1, rfsize*rfisize*3) et le fait rent
    # dans le tableau de patches à la ième ligne
    patches[i]=(patch.reshape(1,-1))
#Normalise les patches 
patches=(patches-patches.mean(1)[:,None])/np.sqrt(patches.var(1)+10)[:,None]


#from sklearn import preprocessing
#patches = preprocessing.normalize(patches, norm='l2')

#Normlize
print(patches[0])
if whitening ==True: 
    [D,V]=np.linalg.eig(np.cov(patches,rowvar=0))

    P = V.dot(np.diag(np.sqrt(1/(D + 0.1)))).dot(V.T)
    patches = patches.dot(P)




Extraction patche: 0 / 200000
Extraction patche: 1000 / 200000
Extraction patche: 2000 / 200000
Extraction patche: 3000 / 200000
Extraction patche: 4000 / 200000
Extraction patche: 5000 / 200000
Extraction patche: 6000 / 200000
Extraction patche: 7000 / 200000
Extraction patche: 8000 / 200000
Extraction patche: 9000 / 200000
Extraction patche: 10000 / 200000
Extraction patche: 11000 / 200000
Extraction patche: 12000 / 200000
Extraction patche: 13000 / 200000
Extraction patche: 14000 / 200000
Extraction patche: 15000 / 200000
Extraction patche: 16000 / 200000
Extraction patche: 17000 / 200000
Extraction patche: 18000 / 200000
Extraction patche: 19000 / 200000
Extraction patche: 20000 / 200000
Extraction patche: 21000 / 200000
Extraction patche: 22000 / 200000
Extraction patche: 23000 / 200000
Extraction patche: 24000 / 200000
Extraction patche: 25000 / 200000
Extraction patche: 26000 / 200000
Extraction patche: 27000 / 200000
Extraction patche: 28000 / 200000
Extraction patche: 29000 / 

In [23]:
## We can also use the run_kmeans I have code below 

kmeans = MiniBatchKMeans(n_clusters=25, n_init=5, max_iter=100, init_size=300, batch_size=1000).fit(patches)
Nrep= kmeans.cluster_centers_
NrepL= list(Nrep)
print(len(Nrep))
array= np.array(Nrep)
centroids=array
print(array.shape)


25
(25, 108)


In [3]:
import random
def run_kmeans(X,k,iterations):
    x2= np.sum(np.power(X,2), axis=1)
   #centroids = valeur alétoire suivant la loi normale 
    centroids= np.random.randn(k, np.size(X,1))*0.1
    
    BATCH_SIZE=1000
    
    for itr in range(1,iterations+1):
        print('kmeans iteration ', itr ,'/', iterations)
        c2= 0.5*np.sum(np.power(centroids,2), axis=1)
        summation= np.zeros((k, np.size(X,1))) 
       
        count1=[]
        loss=0
        c=0
        for i in range(1,len(X), BATCH_SIZE ):
            lastindex=min(i+BATCH_SIZE-1, len(X))
            
            m= lastindex -i +1 
            
            batch= X[i-1:lastindex ,:]  
            batch_t=batch.transpose()
            tmp= centroids.dot(batch_t)
            for n in range(1, len(batch)):
                tmp[:,n]=tmp[:,n]-c2
                
            val=[]
            labels=[]
            [val ,labels] = [np.max(tmp, axis=0),np.argmax(tmp, axis=0) ]   
            
            loss= loss + np.sum(0.5*x2[i-1:lastindex]-val.transpose())
        #transformer label en matrice indicatrice 
            S=np.zeros((m,k))
          
            for j in range(1, lastindex-i):
                S[j][labels[j]]=1
           
            
            summation = summation+(S.transpose()).dot(X[i-1:lastindex,:])
            counts= (np.sum(S,axis=0).transpose())
            count1.append(counts)
          
            c=c+1
           
        counts=np.sum(np.array(count1),axis=0)
        
        for i in range(1, k):
            
            if counts[i].all!=0:
                centroids[i]=summation[i,:]/counts[i]
            elif counts[i]==0:
                centroids[i]=centroids[i]*0
                   
         #on zappe les centroids vide , pour ne pas avoir NaN value   
        badIndex= [i for i,val in enumerate(counts) if val.any()==0]
        centroids[badIndex,:]=0     
            
    return centroids, counts

In [4]:
centroids=run_kmeans(patches,numCentroids,50)



kmeans iteration  1 / 50
kmeans iteration  2 / 50
kmeans iteration  3 / 50
kmeans iteration  4 / 50
kmeans iteration  5 / 50
kmeans iteration  6 / 50
kmeans iteration  7 / 50
kmeans iteration  8 / 50
kmeans iteration  9 / 50
kmeans iteration  10 / 50
kmeans iteration  11 / 50
kmeans iteration  12 / 50
kmeans iteration  13 / 50
kmeans iteration  14 / 50
kmeans iteration  15 / 50
kmeans iteration  16 / 50
kmeans iteration  17 / 50
kmeans iteration  18 / 50
kmeans iteration  19 / 50
kmeans iteration  20 / 50
kmeans iteration  21 / 50
kmeans iteration  22 / 50
kmeans iteration  23 / 50
kmeans iteration  24 / 50
kmeans iteration  25 / 50
kmeans iteration  26 / 50
kmeans iteration  27 / 50
kmeans iteration  28 / 50
kmeans iteration  29 / 50
kmeans iteration  30 / 50
kmeans iteration  31 / 50
kmeans iteration  32 / 50
kmeans iteration  33 / 50
kmeans iteration  34 / 50
kmeans iteration  35 / 50
kmeans iteration  36 / 50
kmeans iteration  37 / 50
kmeans iteration  38 / 50
kmeans iteration  39 

In [5]:
centroids= np.array(centroids[0])


In [6]:
# Generation d'autre patches aléatoire 
patches1= np.zeros((numPatches, rfSize*rfSize*3))
for i in range(0, numPatches):
    if i%1000==0:
        print('Extraction patche:', i, '/', numPatches)
    r= np.random.randint(CIFAR_DIM[0]-rfSize) 
    c= np.random.randint(CIFAR_DIM[0]-rfSize)
    patch= (X_train[(i%len(X_train)),:])  

    patch= patch[:,r:r+rfSize, c:c+rfSize]
  
    patches1[i]=(patch.reshape(1,-1))
patches1=(patches1-patches1.mean(1)[:,None])/np.sqrt(patches1.var(1)+10)[:,None]

#matrice withening
patches1=patches1.dot(P)

#Calcul des distances patches Centroids 
x2=np.power(patches1,2).sum(1)
c2=np.power(centroids,2).sum(1)
xc=patches1.dot(centroids.T)
dist=np.sqrt(-2*xc+x2[:,None]+c2)

#Recuperation des Index Minimum
indexMin=[]
for i in range(0, len(dist)):
    indexMin.append(np.argmin(dist[i]))
    
NewRep= np.zeros((len(X_train),4))
j=0

for i in range(0, len(dist)-4000):
    
    NewRep[i%len(X_train),j]=indexMin[i]
    
    if i== (j+1)*len(X_train):
        j=j+1
#Nouvelle représentation vectorielle 
NewRepBis=np.zeros((len(X_train),4*numCentroids))
for i in range(0, len(NewRep)):
    NewRepBis[i, int(NewRep[i,0])]=1
    NewRepBis[i, numCentroids+int(NewRep[i,1])]=1
    NewRepBis[i, 2*numCentroids+int(NewRep[i,2])]=1
    NewRepBis[i, 3*numCentroids+int(NewRep[i,3])]=1    
   
    

Extraction patche: 0 / 200000
Extraction patche: 1000 / 200000
Extraction patche: 2000 / 200000
Extraction patche: 3000 / 200000
Extraction patche: 4000 / 200000
Extraction patche: 5000 / 200000
Extraction patche: 6000 / 200000
Extraction patche: 7000 / 200000
Extraction patche: 8000 / 200000
Extraction patche: 9000 / 200000
Extraction patche: 10000 / 200000
Extraction patche: 11000 / 200000
Extraction patche: 12000 / 200000
Extraction patche: 13000 / 200000
Extraction patche: 14000 / 200000
Extraction patche: 15000 / 200000
Extraction patche: 16000 / 200000
Extraction patche: 17000 / 200000
Extraction patche: 18000 / 200000
Extraction patche: 19000 / 200000
Extraction patche: 20000 / 200000
Extraction patche: 21000 / 200000
Extraction patche: 22000 / 200000
Extraction patche: 23000 / 200000
Extraction patche: 24000 / 200000
Extraction patche: 25000 / 200000
Extraction patche: 26000 / 200000
Extraction patche: 27000 / 200000
Extraction patche: 28000 / 200000
Extraction patche: 29000 / 

In [7]:
NewRepBis=np.zeros((len(X_train),4*numCentroids))
for i in range(0, len(NewRep)):
    NewRepBis[i, int(NewRep[i,0])]=1
    NewRepBis[i, numCentroids+int(NewRep[i,1])]=1
    NewRepBis[i, 2*numCentroids+int(NewRep[i,2])]=1
    NewRepBis[i, 3*numCentroids+int(NewRep[i,3])]=1

In [8]:
A=NewRepBis

In [49]:
from sklearn import svm

clf = svm.SVC()
clf.fit(A, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
clf.score(A,y_train)

0.22583673469387755

In [7]:
#function Im2col allows to find overllaping patch

def im2col_sliding_broadcasting(A, BSZ, stepsize=1):
    # Parameters
    M,N = A.shape
    col_extent = N - BSZ[1] + 1
    row_extent = M - BSZ[0] + 1

    # Get Starting block indices
    start_idx = np.arange(BSZ[0])[:,None]*N + np.arange(BSZ[1])

    # Get offsetted indices across the height and width of input array
    offset_idx = np.arange(row_extent)[:,None]*N + np.arange(col_extent)

    # Get all actual indices & index into input array for final output
    return np.take (A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

In [8]:
from scipy.spatial import distance_matrix
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
#XC = np.zeros((50000,4*100))
def extract_features (X):
    #(nargin==6)
    trainXC=[]
    liste=[]
    for i in range(0, len(X)):
        if (i%1000==0):
            print('Extraction features: ')#+i+ '/'+ len(X))
            
        #shape de patches (729,108)    
        patches =np.concatenate([im2col_sliding_broadcasting((X[i].reshape(-1,1)[0:1024]).reshape(32,32) , [6,6], stepsize=1)
        ,im2col_sliding_broadcasting((X[i].reshape(-1,1)[1024:2048]).reshape(32,32), [6,6], stepsize=1),
        im2col_sliding_broadcasting((X[i].reshape(-1,1)[2048:]).reshape(32,32),[ 6,6], stepsize=1) ], axis=0)
        patches=patches.transpose()
        
        patches=np.array(patches)
        
        
        patches=(patches-patches.mean(1)[:,None])/(np.sqrt(patches.var(1)+10)[:,None])
        #map to feature space
        if whitening==True:
            patches=patches.dot(P)
            
            
        #calculate distance using x2-2xc+c2
        x2=np.power(patches,2).sum(1)
        c2=np.power(centroids,2).sum(1)
        xc=patches.dot(centroids.T)
        dist=np.sqrt(-2*xc+x2[:,None]+c2)
        
       #patches.dot(P) si matrice
      
        
        
        z =dist # 13h          
      
        
        u=dist.mean(1)
        patches=np.maximum(-dist+u[:,None],0)
        rs=CIFAR_DIM[0]-rfSize+1
        cs=CIFAR_DIM[1]-rfSize+1
        patches=np.reshape(patches,[rs,cs,-1])
        q=[]
        q.append(patches[0:int(rs/2)+1,0:int(cs/2)+1].sum(0).sum(0))
        q.append(patches[0:int(rs/2)+1,int(cs/2)+1:cs-1].sum(0).sum(0))
        q.append(patches[int(rs/2)+1:rs-1,0:int(cs/2)+1].sum(0).sum(0))
        q.append(patches[int(rs/2)+1:rs-1,int(cs/2)+1:cs-1].sum(0).sum(0))
        q=np.array(q).ravel()
        trainXC.append(q)
    trainXC=np.array(trainXC)
    trainXC=(trainXC-trainXC.mean(1)[:,None])/(np.sqrt(trainXC.var(1)+.01)[:,None])
    return trainXC
        
        
        
        
        
        

In [9]:
### Calcul of features , with matrices distances ,take distance < mean(distance) + pooling patches

#Extract features of Xtrain




A=extract_features (X_train )

print(np.shape(A[2]))

Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction features: 
Extraction

In [10]:
#Extract features of X_test 
print(A.shape)

BBB=extract_features (X_val )

(49000, 100)
Extraction features: 


In [9]:
#train svm classifier 

import numpy as np
def my_l2svmloss(w, *args):
    #print(len(args))
    X=args[0]
    y=args[1]
    K=args[2]
    C=args[3]
    [M,N]= np.shape(X)   
    theta= w.reshape(N,K)  
    Y= np.zeros((len(y),K))   
    for i in range(0, len(y)):
        for k in range(1, K+1):
            if y[i]==k:
                Y[i,k-1]=Y[i,k-1]+1       
            else: Y[i,k-1]=Y[i,k-1]-1

    margin = np.maximum(0, 1-Y*(X.dot(theta)))    
  
    loss= (0.5* np.sum((np.power(theta,2)), axis=0))+ C*np.mean((np.power(margin,2)), axis=0)
  
    loss= np.sum(loss, axis=0) 
  
    g = theta -2*C/M*(X.transpose().dot(margin*Y))
    g.reshape(np.size(X, 1)*K,1)
  
    return loss, np.hstack(g)

#objectif mininimiser loss et g 


from scipy.optimize import fmin_l_bfgs_b
def train_svm(trainXC, trainY, C):
    X= trainXC   #featurize data 
    y= trainY   
    
    K = np.max(trainY)  #number of classes 
    
    w0= np.zeros((np.size(X, 1)*K,1) )# 10* (4*numcentroids+1) , w vector of weight for the 10 classes 
    w0=np.array(w0)
    #minimize the function svmloss
    w, fw ,i   = fmin_l_bfgs_b(my_l2svmloss,w0,args =([X,y,K,C  ]), maxfun=1000, maxiter=1000)  #x,f,d   / w, fw, i
   
    theta= w.reshape(np.size(trainXC, 1), K )
    return theta 
    
    
    
    


In [10]:
#from sklearn import preprocessing
import numpy as np
#Asc = preprocessing.scale(A)

#Standardize the data 
#Asc= A- np.mean(A,axis=0)
#Asc= Asc/np.sqrt(np.var(A,axis=0)+0.01)
#One= np.array([1]*49000)

Asc=np.c_[ A, np.ones(len(A)) ] 

#train SVM with with the features of A (features from Xtrain) standardised , and Y_train=Cifar[1]+1

theta=train_svm(Asc, y_train+1 , 100)

In [11]:
# we find with f_min the function who find w  who minimize the fonction svmloss

Matrice = Asc.dot(theta)      #theta= w  theta shape : (4* num Centroids, number of class ), Asc= x ,
#Asc.shape = 50000, 4*numCentroids, Matrice (50000, 10) argamx axis= 1 sort the number of the class predict 
#higher score with weight vector 
[val1 ,inds1] = [np.max(Matrice, axis=1),np.argmax(Matrice, axis=1) ]  #y_pred= argmax(x*wy)
print('train result :')
(inds1==y_train).mean()

train result :


0.26816326530612244

In [14]:
#Standardize the data of X_test with the data X_train
reserve=BBB
BBB=BBB- np.mean(A,axis=0)
BBB= BBB/ np.sqrt(np.var(A,axis=0)+0.01)
Bsc=np.c_[ BBB, np.ones(len(BBB)) ] 

In [15]:
MatriceTest= Bsc.dot(theta)
[val2 ,inds2] = [np.max(MatriceTest, axis=1),np.argmax(MatriceTest, axis=1) ]
print('Accuraccy result on Xtest:')
(inds2==y_val).mean()

Accuraccy result on Xtest:


0.468

In [16]:
test=extract_features (X_test )
test=test- np.mean(A,axis=0)
test= test/ np.sqrt(np.var(A,axis=0)+0.01)
Tsc=np.c_[test, np.ones(len(test)) ] 
MatriceTest1= Tsc.dot(theta)
[val3 ,inds3] = [np.max(MatriceTest1, axis=1),np.argmax(MatriceTest1, axis=1) ]
print('Accuraccy result on Xtest:')
(inds3==y_test).mean()

Extraction features: 
Accuraccy result on Xtest:


0.449

In [17]:
import itertools
%matplotlib
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

y_test = y_test
y_pred = inds3


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['airplane', 'automobile', 'bird', 'cat', 'deer' ,'dog', 'frog', 'horse' ,'ship' ,'truck'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['airplane', 'automobile', 'bird', 'cat', 'deer' ,'dog', 'frog', 'horse' ,'ship' ,'truck'], normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Using matplotlib backend: TkAgg
Confusion matrix, without normalization
[[42  9  1  2  2  4  5  8 24  6]
 [ 4 55  2  1  2  1  5  0 10  9]
 [17  2 13 12  8 12 15 13  2  6]
 [ 2  6  4 30  4 21 13 10  3 10]
 [ 8  5  7  7 23  5 11 17  4  3]
 [ 0  4  5 18  5 35 10  8  0  1]
 [ 1  4  5  4  5  6 72 10  1  4]
 [ 2  4  1  5  3 14  4 64  2  3]
 [23 11  0  2  1  1  1  1 61  5]
 [ 5 18  0  2  1  3 11  5 10 54]]
Normalized confusion matrix
[[0.41 0.09 0.01 0.02 0.02 0.04 0.05 0.08 0.23 0.06]
 [0.04 0.62 0.02 0.01 0.02 0.01 0.06 0.   0.11 0.1 ]
 [0.17 0.02 0.13 0.12 0.08 0.12 0.15 0.13 0.02 0.06]
 [0.02 0.06 0.04 0.29 0.04 0.2  0.13 0.1  0.03 0.1 ]
 [0.09 0.06 0.08 0.08 0.26 0.06 0.12 0.19 0.04 0.03]
 [0.   0.05 0.06 0.21 0.06 0.41 0.12 0.09 0.   0.01]
 [0.01 0.04 0.04 0.04 0.04 0.05 0.64 0.09 0.01 0.04]
 [0.02 0.04 0.01 0.05 0.03 0.14 0.04 0.63 0.02 0.03]
 [0.22 0.1  0.   0.02 0.01 0.01 0.01 0.01 0.58 0.05]
 [0.05 0.17 0.   0.02 0.01 0.03 0.1  0.05 0.09 0.5 ]]


In [ ]:
input_size = A.shape[1]
hidden_size = 200
num_classes = 10

Asc1= A- np.mean(A,axis=0)
Asc1= Asc1/np.sqrt(np.var(A,axis=0)+0.01)
net = TwoLayerNet(input_size, hidden_size, num_classes,1e-4)
stats = net.train(Asc1, Cifar[1] , BBB, Cifar[3] ,
                            num_iters=70000, batch_size=128,
                            learning_rate=5e-4, learning_rate_decay=0.99,
                            reg=0, verbose=True,update="SGD",arg=0.95,dropout=0.3)

In [50]:
val_acc = (net.predict(Asc1) == Cifar[1]).mean()


In [51]:
val_acc

0.29787999999999998

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout , Activation
from keras.optimizers import RMSprop
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sklearn.preprocessing
from keras.optimizers import SGD
#optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon = 1e-06, decay=1e-6)

a=y_train
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(a)+1))
C1=label_binarizer.transform(a)
b=y_val
C2=label_binarizer.transform(b)
model= Sequential()
model.add(Dense(50,input_shape=(A.shape[1],)))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
#sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon = 1e-06, decay=1e-6), metrics=['accuracy'])
history = model.fit(A, C1 , batch_size=1000, epochs=1000, verbose=2, validation_split=0.2)
loss_and_metrics= model.evaluate(BBB, C2)
print(loss_and_metrics)

In [21]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [22]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model1 = Sequential()
model1.add(Dense(64, input_dim=20, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model1.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
from scipy.spatial import distance_matrix
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance
#XC = np.zeros((50000,4*100))
def extract_features (X):
    whitening = True #(nargin==6)
    trainXC=[]
    liste=[]
    for i in range(0, len(X)):
        if (i%1000==0):
            print('Extraction features: ')#+i+ '/'+ len(X))
            
        #shape de patches (729,108)    
        patches =np.concatenate([im2col_sliding_broadcasting((X[i].reshape(-1,1)[0:1024]).reshape(32,32) , [6,6], stepsize=1)
        ,im2col_sliding_broadcasting((X[i].reshape(-1,1)[1024:2048]).reshape(32,32), [6,6], stepsize=1),
        im2col_sliding_broadcasting((X[i].reshape(-1,1)[2048:]).reshape(32,32),[ 6,6], stepsize=1) ], axis=0)
        patches=patches.transpose()
        
        patches=np.array(patches)
        
        NumCentroids= np.size(Centroids,0)
        
        patches=(patches-patches.mean(1)[:,None])/(np.sqrt(patches.var(1)+10)[:,None])
        #map to feature space
        patches=patches.dot(P)
        #calculate distance using x2-2xc+c2
        x2=np.power(patches,2).sum(1)
        c2=np.power(centroids,2).sum(1)
        xc=patches.dot(centroids.T)
        dist=np.sqrt(-2*xc+x2[:,None]+c2)
        
       #patches.dot(P) si matrice
      
        
        
        z =dist # 13h          
      
        
        u=dist.mean(1)
        patches=np.maximum(-dist+u[:,None],0)
        rs=CIFAR_DIM[0]-rfSize+1
        cs=CIFAR_DIM[1]-rfSize+1
        patches=np.reshape(patches,[rs,cs,-1])
        q=[]
        q.append(patches[0:int(rs/2)+1,0:int(cs/2)+1].sum(0).sum(0))
        q.append(patches[0:int(rs/2)+1,int(cs/2)+1:cs-1].sum(0).sum(0))
        q.append(patches[int(rs/2)+1:rs-1,0:int(cs/2)+1].sum(0).sum(0))
        q.append(patches[int(rs/2)+1:rs-1,int(cs/2)+1:cs-1].sum(0).sum(0))
        q=np.array(q).ravel()
        trainXC.append(q)
        
    
        
        
    trainXC=np.array(trainXC)
    trainXC=(trainXC-trainXC.mean(1)[:,None])/(np.sqrt(trainXC.var(1)+.01)[:,None])
   
        
        
        
        
    return trainXC


for line in trainXC:
    line= np.split(line, 4)
    indiceMax=[]
    j=0
    for e in line:
        indiceMax.append(argmax(e)+j)
        j=j+numCentroids
    for el not in indiceMax:
        trainXC[line, el]=0
    for el in indiceMax:
        trainXC[line, el]=1





